# Database from
https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k/data?select=full_df.csv

In [47]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [48]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

from imblearn.under_sampling import RandomUnderSampler

from helpers.help import *
from helpers.helptf import *
from sklearn.utils import resample

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input

# Read dataset
Reading both eyes while filtering the intended diseases
and including the augmented images

In [ ]:
# load dataset
df = pd.read_csv('hr-dataset/full_df.csv')

#Left eye
# get the diagnostic of hypertensive retinopathy
ds_hr_left = df[df['Left-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]
# get the diagnostic of diabetic retinopathy
ds_dr_left = df[df['Left-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]
# get the diagnostic for no rethinopathy
ds_nr_left = df[~df['Left-Diagnostic Keywords'].str.contains('retinopathy', na=False)]
# get the diagnostic of normal fundus
# ds_normal_left = df[df['Left-Diagnostic Keywords'] == 'normal fundus']


#Right eye
# get the diagnostic of hypertensive retinopathy
ds_hr_right = df[df['Right-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]
# get the diagnostic of diabetic retinopathy
ds_dr_right = df[df['Right-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]
# get the diagnostic for no retinopathy
ds_nr_right = df[~df['Right-Diagnostic Keywords'].str.contains('retinopathy', na=False)]
# get the diagnostic of normal fundus
# ds_normal_right = df[df['Right-Diagnostic Keywords'] == 'normal fundus']



# Specific dataframe
# Left eye
df_hr_left = ds_hr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_dr_left = ds_dr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_nr_left = ds_nr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]

# Right eye
df_hr_right = ds_hr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]
df_dr_right = ds_dr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]
df_nr_right = ds_nr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]


# Droping class
# Left eye
df_hr_left = df_hr_left.drop('Left-Diagnostic Keywords', axis=1)
df_dr_left = df_dr_left.drop('Left-Diagnostic Keywords', axis=1)
df_nr_left = df_nr_left.drop('Left-Diagnostic Keywords', axis=1)
# Right eye
df_hr_right = df_hr_right.drop('Right-Diagnostic Keywords', axis=1)
df_dr_right = df_dr_right.drop('Right-Diagnostic Keywords', axis=1)
df_nr_right = df_nr_right.drop('Right-Diagnostic Keywords', axis=1)



# Get augmented images
path = os.path.join(os.getcwd(),'hr-dataset/augmented_images')
list_hr_augmented = []
list_dr_augmented = []

for file in os.listdir(path + '/hr'):
    list_hr_augmented.append(file)
    
for file in os.listdir(path + '/dr'):
    list_dr_augmented.append(file)
    
df_hr_augmented = pd.DataFrame(list_hr_augmented, columns = ['Left-Fundus'])
df_dr_augmented = pd.DataFrame(list_dr_augmented, columns = ['Left-Fundus'])





print("---- LEFT ----")
print(df_hr_left.shape[0])
print(df_dr_left.shape[0])
print(df_nr_left.shape[0])
print("---- RIGHT ----")
print(df_hr_right.shape[0])
print(df_dr_right.shape[0])
print(df_nr_right.shape[0])
print("---- AUGMENTED ----")
print(df_hr_augmented.shape[0])
print(df_dr_augmented.shape[0])


---- LEFT ----
191
85
4587
---- RIGHT ----
191
80
4511
---- AUGMENTED ----
1153
480


# Solving the undersampling of HR 

In [ ]:

# join left and right eye sets
df_hr = pd.concat([df_hr_left, df_hr_right, df_hr_augmented ])
df_dr = pd.concat([df_dr_left, df_dr_right, df_dr_augmented ])
df_nr = pd.concat([df_nr_left, df_nr_right])


"""
    Since the minimum value of samples is from DR: 645 (480+164), we will undersample the rest
    to balance the dataset
"""

df_hr_downsampled = resample(df_hr, replace=False, n_samples=690, random_state=10)
df_dr_downsampled = resample(df_dr, replace=False, n_samples=645, random_state=10)
df_nr_downsampled = resample(df_nr, replace=False, n_samples=720, random_state=10)

print(df_hr_downsampled.shape[0])
print(df_dr_downsampled.shape[0])
print(df_nr_downsampled.shape[0])

690
645
720


# Class transformation

In [ ]:
# Prepare dataset paths
path = os.path.join(os.getcwd(),'hr-dataset/preprocessed_images')
pathHR = os.path.join(os.getcwd(),'hr-dataset/augmented_images/hr')
pathDB = os.path.join(os.getcwd(),'hr-dataset/augmented_images/dr')

# 0 - No Diabetic or Hipertensive Retinopathy
# 1 - Diabetic Retinopathy
# 2 - Hipertensive Retinopathy

array = []
detailPath = ""

# get HR images
for index, row in df_hr_downsampled.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
        detailPathHR = os.path.join(pathHR,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        array.append([detailPath,2])
    elif(os.path.exists(detailPathHR)):
        array.append([detailPathHR,2])



# get DR images
for index, row in df_dr_downsampled.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
        detailPathDR = os.path.join(pathDB,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        array.append([detailPath,1])
    elif(os.path.exists(detailPathDR)):
        array.append([detailPathDR,1])


# get no rethinopaty images
for index, row in df_nr_downsampled.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        array.append([detailPath,0])


    
# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

2028

# Get the data ready
Separating 10% of the data for test and 11% for validation

In [ ]:
X,y=dataset[::,0],dataset[::,1]
y = y.astype(int)

#One hot encode the labels
y = to_categorical(y)


#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 10% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.1*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]

#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 11% 
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.11, random_state=42)

In [53]:
print(f"Samples in Training set: {x_train.shape[0]}")
print(f"Samples in Validation set: {x_val.shape[0]}")
print(f"Samples in Test set: {x_test.shape[0]}")

Samples in Training set: 1625
Samples in Validation set: 201
Samples in Test set: 202


In [54]:
# Check if imbalance
for i in [y_train, y_test, y_val]:
    print(np.unique(i, return_counts = True, axis = 0))

(array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([539, 522, 564]))
(array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([78, 57, 67]))
(array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([73, 61, 67]))


# Prepares Data for the model
Create batches of 32 for validation and testing sets
Batch of 16 for training set

In [55]:
val_dataset=build_dataset(x_val,y_val,repeat=False,batch=32)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=32)

BATCH_SIZE=16
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_train,y_train,batch=BATCH_SIZE)

# input shape for the model
input_shape=train_dataset.element_spec[0].shape[1:]


print(train_dataset)
print(val_dataset)
print(test_dataset)

input_shape=train_dataset.element_spec[0].shape[1:]
print(input_shape)

for batch in train_dataset.take(1):
    features, labels = batch  # Unpack the tuple
    print(features.shape[0])  # Number of elements in the batch
    print(labels.shape[0])  # Number of elements in the batch

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
(224, 224, 3)
16
16


# Load model

In [56]:
dr_model = load_model('model/model_al.keras')

dr_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 110, 110, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 110, 110, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 53, 53, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 53, 53, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,510,472 (127.83 MB)

 Trainable params: 11,169,922 (42.61 MB)

 Non-trainable params: 704 (2.75 KB)

 Optimizer params: 22,339,846 (85.22 MB)

# Use the transfer learning pre-built function
Will cut the 10 upper layers, and 3 output classes

In [57]:
# classes: {"no_rethinopathy", "dr","hr"}
transfer_model = prep_translearn(model=dr_model, top_layers_to_cut=10, out_dim=3, learning_rate=0.0001) 

After layer 0 (conv2d), shape: (None, 224, 224, 32)
After layer 1 (batch_normalization), shape: (None, 224, 224, 32)
After layer 2 (max_pooling2d), shape: (None, 112, 112, 32)
After layer 3 (dropout), shape: (None, 112, 112, 32)
After layer 4 (conv2d_1), shape: (None, 110, 110, 64)
After layer 5 (batch_normalization_1), shape: (None, 110, 110, 64)
After layer 6 (max_pooling2d_1), shape: (None, 55, 55, 64)


# Configure transfer learning model

In [58]:
transfer_model.compile(
        loss = "categorical_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )

transfer_model.summary()

Model: "functional_71"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ NEW_input (InputLayer)          │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 110, 110, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 110, 110, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_FLAT                        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_Signature (Dense)           │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_Dropout (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_output (Dense)              │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,483 (111.26 KB)

 Trainable params: 28,291 (110.51 KB)

 Non-trainable params: 192 (768.00 B)

# Train model

In [59]:
# saves the model with the lowest validation Loss
checkpoint=ModelCheckpoint(filepath='model/model_transferlearning_aug.keras',
                           monitor='val_loss',save_best_only=True,verbose=1)

# logs the training progress to a CSV
csv_logger=keras.callbacks.CSVLogger('logger/trainlog_transferlearning_aug.csv',
                                     separator=',',append=False)

# defines a early stop if in 10 epoches the validation loss dont improve
early_stopper=keras.callbacks.EarlyStopping(monitor='val_loss',
                                            min_delta=0.001,
                                            restore_best_weights=True,
                                            patience=10)

callbacks_list=[checkpoint,early_stopper,csv_logger]

In [ ]:
EPOCHS = 100
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

transfer_model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=EPOCHS,
          validation_data=val_dataset,validation_steps=None,
          callbacks=callbacks_list)

Epoch 1/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - categorical_accuracy: 0.4403 - loss: 1.0643
Epoch 1: val_loss improved from inf to 0.95706, saving model to model/model_transferlearning_aug.keras
101/101 ━━━━━━━━━━━━━━━━━━━━ 35s 341ms/step - categorical_accuracy: 0.4407 - loss: 1.0637 - val_categorical_accuracy: 0.4876 - val_loss: 0.9571
Epoch 2/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - categorical_accuracy: 0.5138 - loss: 0.9483
Epoch 2: val_loss did not improve from 0.95706
101/101 ━━━━━━━━━━━━━━━━━━━━ 33s 331ms/step - categorical_accuracy: 0.5140 - loss: 0.9480 - val_categorical_accuracy: 0.4080 - val_loss: 1.3957
Epoch 3/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step - categorical_accuracy: 0.5585 - loss: 0.8984
Epoch 3: val_loss did not improve from 0.95706
101/101 ━━━━━━━━━━━━━━━━━━━━ 35s 348ms/step - categorical_accuracy: 0.5584 - loss: 0.8984 - val_categorical_accuracy: 0.4577 - val_loss: 1.1076
Epoch 4/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - categorical

# Evaluation

In [61]:
# load the best model, trained before
model = keras.models.load_model("model/model_transferlearning_aug.keras")
print("-" * 100)

# evaluates with the test_dataset
print(model.evaluate(test_dataset, verbose=0,return_dict=True))

----------------------------------------------------------------------------------------------------
{'categorical_accuracy': 0.7227723002433777, 'loss': 0.6367489099502563}


Testing

In [ ]:
class_names = {0: 'No Diabetic or Hipertensive Retinopathy', 1: 'Diabetic Retinopathy', 2: 'Hipertensive Retinopathy'}

y_pred = []
y_true = []
y_proba = []

# Iterate over the test dataset
for x_batch, y_batch in test_dataset:
    # Predict probabilities for each batch
    y_test_proba = transfer_model.predict(x_batch)

    # Convert probabilities to predicted class labels (0, 1, or 2)
    y_pred.extend(np.argmax(y_test_proba, axis=1))

    y_proba.extend(y_test_proba)

    # Convert true labels from one-hot encoding to class labels (0, 1, or 2)
    y_true.extend(np.argmax(y_batch.numpy(), axis=1))

# Convert lists to numpy arrays
y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Transform numerical labels into class names
y_pred_names = [class_names[label] for label in y_pred]
y_true_names = [class_names[label] for label in y_true]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


Confusion matrix and Classification report

In [63]:
conf_matrix = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

# Print classification report
print("\nClassification Report:\n", classification_report(y_true_names, y_pred_names))


Confusion Matrix:
 [[62  1  4]
 [12 20 25]
 [ 9  5 64]]

Classification Report:
                           precision    recall  f1-score   support

    Diabetic Retinopathy       0.77      0.35      0.48        57
Hipertensive Retinopathy       0.69      0.82      0.75        78
                  Normal       0.75      0.93      0.83        67

                accuracy                           0.72       202
               macro avg       0.73      0.70      0.69       202
            weighted avg       0.73      0.72      0.70       202



# AUC

In [64]:
# Convert true labels (y_true) from integers to one-hot encoding
y_true_one_hot = tf.keras.utils.to_categorical(y_true, num_classes=3)

# Get predicted probabilities for each class (y_test_proba)
y_proba = np.array(y_proba)  


# Calculate the ROC-AUC score for each class
auc = roc_auc_score(y_true_one_hot, y_proba, average='macro', multi_class='ovr')

print(f"Macro-Averaged ROC-AUC Score: {auc:.4f}")

Macro-Averaged ROC-AUC Score: 0.8754
